# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [16]:
import os
import sys

sys.path.append('..')
from utils import *

# Set your preferred name
USER = "Arthur"
# Set the data and output directories
DATA_ROOT = 'C:/Users/arthu/Downloads'
OUTPUT_ROOT = 'C:/Users/arthu/OneDrive/Documents/GitHub/output2'
TRAIT = "Anorexia-Nervosa"

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

# 2. Data preprocessing and selection

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [13]:
dataset = 'GEO'
trait_subdir = "Anorexia-Nervosa"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE60190']

In [18]:
cohort = accession_num = "GSE60190"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('C:/Users/arthu/Downloads\\GEO\\Anorexia-Nervosa\\GSE60190\\GSE60190_family.soft.gz',
 'C:/Users/arthu/Downloads\\GEO\\Anorexia-Nervosa\\GSE60190\\GSE60190_series_matrix.txt.gz')

### Initial filtering and clinical data preprocessing

In [19]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Genetic Neuropathology of Obsessive Psychiatric Syndromes"
!Series_summary	"Anorexia nervosa (AN), bulimia nervosa (BN), and obsessive-compulsive disorder (OCD) are complex psychiatric disorders with shared obsessive features, thought to arise from the interaction of multiple genes of small effect with environmental factors.  Potential candidate genes for AN, BN, and OCD have been identified through clinical association and neuroimaging studies; however, recent genome-wide association studies of eating disorders (ED) so far have failed to report significant findings. Additionally, few if any studies have interrogated postmortem brain tissue for evidence of eQTLs associated with candidate genes, which has particular promise as an approach to elucidating molecular mechanisms of association. We therefore selected single nucleotide polymorphisms (SNPs) based on candidate gene studies for AN, BN, and OCD from the literature, and examined the association of these SNPs with gen

In [30]:
clinical_data.head(10)

,!Sample_geo_accession,GSM1467273,GSM1467274,GSM1467275,GSM1467276,GSM1467277,GSM1467278,GSM1467279,GSM1467280,GSM1467281,...,GSM1467396,GSM1467397,GSM1467398,GSM1467399,GSM1467400,GSM1467401,GSM1467402,GSM1467403,GSM1467404,GSM1467405
0,!Sample_characteristics_ch1,rin: 7.4,rin: 8.6,rin: 7.8,rin: 8.2,rin: 8.5,rin: 8.3,rin: 8.1,rin: 8.8,rin: 8.7,...,rin: 8.4,rin: 8,rin: 8.3,rin: 7.5,rin: 8.1,rin: 8.5,rin: 6.6,rin: 7.1,rin: 9,rin: 8.3
1,!Sample_characteristics_ch1,ocd: ED,ocd: Control,ocd: OCD,ocd: Control,ocd: Control,ocd: ED,ocd: Control,ocd: Control,ocd: Control,...,ocd: ED,ocd: Control,ocd: Control,ocd: Control,ocd: Control,ocd: Control,ocd: ED,ocd: Control,ocd: Control,ocd: Control
2,!Sample_characteristics_ch1,rinmatched: 1,rinmatched: 0,rinmatched: 1,rinmatched: 0,rinmatched: 0,rinmatched: 1,rinmatched: 0,rinmatched: 0,rinmatched: 0,...,rinmatched: 1,rinmatched: 1,rinmatched: 0,rinmatched: 1,rinmatched: 1,rinmatched: 0,rinmatched: 1,rinmatched: 1,rinmatched: 0,rinmatched: 0
3,!Sample_characteristics_ch1,dx: Bipolar,dx: Control,dx: Bipolar,dx: Control,dx: Control,dx: Bipolar,dx: Control,dx: Control,dx: Control,...,dx: ED,dx: Control,dx: Control,dx: Control,dx: Control,dx: Control,dx: MDD,dx: Control,dx: Control,dx: Control
4,!Sample_characteristics_ch1,ph: 6.18,ph: 6.59,ph: 6.37,ph: 6.6,ph: 6.38,ph: 6.02,ph: 6.87,ph: 6.95,ph: 6.82,...,ph: 6.34,ph: 6.5,ph: 6.51,ph: 6.59,ph: 6.65,ph: 6.71,ph: 6.2,ph: 6.2,ph: 6.83,ph: 6.92
5,!Sample_characteristics_ch1,age: 50.421917,age: 27.49863,age: 30.627397,age: 61.167123,age: 32.69589,age: 39.213698,age: 58.605479,age: 49.2,age: 41.041095,...,age: 30.726027,age: 63.471232,age: 54.808219,age: 57.512328,age: 57.610958,age: 44.958904,age: 35.684931,age: 63,age: 38.780821,age: 45.978082
6,!Sample_characteristics_ch1,pmi: 27,pmi: 19.5,pmi: 71.5,pmi: 19.5,pmi: 22.5,pmi: 22.5,pmi: 64,pmi: 28,pmi: 18,...,pmi: 19,pmi: 37,pmi: 21,pmi: 24,pmi: 17,pmi: 20,pmi: 33,pmi: 45.5,pmi: 29,pmi: 34
7,!Sample_characteristics_ch1,Sex: F,Sex: M,Sex: M,Sex: M,Sex: M,Sex: F,Sex: M,Sex: M,Sex: M,...,Sex: F,Sex: M,Sex: M,Sex: M,Sex: M,Sex: M,Sex: F,Sex: M,Sex: M,Sex: M
8,!Sample_characteristics_ch1,race: CAUC,race: CAUC,race: CAUC,race: CAUC,race: CAUC,race: CAUC,race: CAUC,race: CAUC,race: CAUC,...,race: CAUC,race: CAUC,race: CAUC,race: CAUC,race: CAUC,race: CAUC,race: CAUC,race: CAUC,race: CAUC,race: CAUC
9,!Sample_characteristics_ch1,batch1: 16,batch1: 18,batch1: 18,batch1: 18,batch1: 19,batch1: 19,batch1: 20,batch1: 21,batch1: 21,...,batch1: 96,batch1: 96,batch1: 97,batch1: 98,batch1: 99,batch1: 99,batch1: 100,batch1: 102,batch1: 102,batch1: 102


In [21]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['rin: 7.4',
  'rin: 8.6',
  'rin: 7.8',
  'rin: 8.2',
  'rin: 8.5',
  'rin: 8.3',
  'rin: 8.1',
  'rin: 8.8',
  'rin: 8.7',
  'rin: 7.5',
  'rin: 9',
  'rin: 7.1',
  'rin: 7.2',
  'rin: 7.7',
  'rin: 8.9',
  'rin: 6.7',
  'rin: 6',
  'rin: 8.4',
  'rin: 7.3',
  'rin: 8',
  'rin: 9.1',
  'rin: 7.9',
  'rin: 9.7',
  'rin: 9.2',
  'rin: 6.5',
  'rin: 7',
  'rin: 7.6',
  'rin: 6.6',
  'rin: 5.4',
  'rin: 5.6'],
 1: ['ocd: ED', 'ocd: Control', 'ocd: OCD'],
 2: ['rinmatched: 1', 'rinmatched: 0'],
 3: ['dx: Bipolar', 'dx: Control', 'dx: MDD', 'dx: Tics', 'dx: OCD', 'dx: ED'],
 4: ['ph: 6.18',
  'ph: 6.59',
  'ph: 6.37',
  'ph: 6.6',
  'ph: 6.38',
  'ph: 6.02',
  'ph: 6.87',
  'ph: 6.95',
  'ph: 6.82',
  'ph: 6.27',
  'ph: 6.53',
  'ph: 6.55',
  'ph: 6',
  'ph: 6.13',
  'ph: 6.08',
  'ph: 6.29',
  'ph: 6.98',
  'ph: 5.91',
  'ph: 6.06',
  'ph: 6.9',
  'ph: 6.83',
  'ph: 6.36',
  'ph: 6.84',
  'ph: 6.74',
  'ph: 6.28',
  'ph: 6.49',
  'ph: 6.7',
  'ph: 6.63',
  'ph: 6.48',
  'ph: 6.62'],
 

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [22]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Anorexia-Nervosa\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Anorexia-Nervosa\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The ke

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [23]:
age_row = gender_row = None
convert_age = convert_gender = None

In [24]:
is_gene_availabe = True
trait_row = 1
age_row = 5
gender_row = 7

trait_type = 'binary'

In [25]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [28]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'ocd: ED':
        return 1  
    else:
        return 0 

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except (IndexError, ValueError):
        return None  # Convert unknown values to None


def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'Sex: F':
        return 0
    elif gender_string == 'Sex: M':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'ocd: ED'
example_age = 'age: 63'
example_gender = 'Sex: M'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(1, 63.0, 1)

In [29]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM1467273,GSM1467274,GSM1467275,GSM1467276,GSM1467277,GSM1467278,GSM1467279,GSM1467280,GSM1467281,GSM1467282,...,GSM1467396,GSM1467397,GSM1467398,GSM1467399,GSM1467400,GSM1467401,GSM1467402,GSM1467403,GSM1467404,GSM1467405
Anorexia-Nervosa,1.000000,0.00000,0.000000,0.000000,0.00000,1.000000,0.000000,0.0,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000
Age,50.421917,27.49863,30.627397,61.167123,32.69589,39.213698,58.605479,49.2,41.041095,51.750684,...,30.726027,63.471232,54.808219,57.512328,57.610958,44.958904,35.684931,63.0,38.780821,45.978082
Gender,0.000000,1.00000,1.000000,1.000000,1.00000,0.000000,1.000000,1.0,1.000000,1.000000,...,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.000000,1.000000


### Genetic data preprocessing and final filtering

In [31]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM1467273,GSM1467274,GSM1467275,GSM1467276,GSM1467277,GSM1467278,GSM1467279,GSM1467280,GSM1467281,GSM1467282,...,GSM1467396,GSM1467397,GSM1467398,GSM1467399,GSM1467400,GSM1467401,GSM1467402,GSM1467403,GSM1467404,GSM1467405
ID,,,,,,,,,,,,,,,,,,,,,
ILMN_1343291,14.631928,14.846268,14.949614,15.044821,14.790150,14.670395,14.730323,14.972534,14.826768,14.790150,...,14.749395,14.593207,14.558687,14.490661,14.540732,14.540732,14.972534,14.730323,14.790150,14.609839
ILMN_1343295,13.015539,13.199489,13.256768,13.030246,12.611440,13.030246,12.894845,12.668329,12.536053,12.684084,...,12.729559,12.628114,12.729559,12.526797,12.921417,12.536053,12.616903,12.685365,12.729559,12.894845
ILMN_1651199,7.407451,7.404480,7.384722,7.398199,7.388903,7.392256,7.411295,7.392441,7.402658,7.397018,...,7.386564,7.392892,7.387466,7.398703,7.387851,7.380980,7.391920,7.391737,7.384270,7.391287
ILMN_1651209,7.551069,7.489157,7.481180,7.454187,7.468150,7.538856,7.516444,7.428403,7.420693,7.435272,...,7.492618,7.517797,7.570776,7.491987,7.543178,7.536219,7.486087,7.445162,7.523485,7.524602
ILMN_1651210,7.389822,7.396480,7.392958,7.389106,7.399044,7.401760,7.412553,7.418004,7.383820,7.399614,...,7.383362,7.386934,7.383795,7.390598,7.383517,7.380075,7.383755,7.392136,7.383855,7.395560


In [33]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['ILMN_1343291',
 'ILMN_1343295',
 'ILMN_1651199',
 'ILMN_1651209',
 'ILMN_1651210',
 'ILMN_1651221',
 'ILMN_1651228',
 'ILMN_1651229',
 'ILMN_1651230',
 'ILMN_1651232',
 'ILMN_1651235',
 'ILMN_1651236',
 'ILMN_1651237',
 'ILMN_1651238',
 'ILMN_1651249',
 'ILMN_1651253',
 'ILMN_1651254',
 'ILMN_1651259',
 'ILMN_1651260',
 'ILMN_1651262']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [34]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209', 'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253', 'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262']\n"


If not required, jump directly to the gene normalization step

In [35]:
requires_gene_mapping = True

In [36]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['ILMN_1725881', 'ILMN_1910180', 'ILMN_1804174', 'ILMN_1796063', 'ILMN_1811966'], 'nuID': ['rp13_p1x6D80lNLk3c', 'NEX0oqCV8.er4HVfU4', 'KyqQynMZxJcruyylEU', 'xXl7eXuF7sbPEp.KFI', '9ckqJrioiaej9_ajeQ'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'Unigene', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'Transcript': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'ILMN_Gene': ['LOC23117', 'HS.575038', 'FCGR2B', 'TRIM44', 'LOC653895'], 'Source_Reference_ID': ['XM_933824.1', 'Hs.575038', 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'RefSeq_ID': ['XM_933824.1', nan, 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'Unigene_ID': [nan, 'Hs.575038', nan, nan, nan], 'Entrez_Gene_ID': [23117.0, nan, 2213.0, 54765.0, 653895.0], 'GI': [89040007.0, 10437021.0, 88952550.0, 29029528.0, 89033487.0], 'Accession': ['XM_933824.1', 'AK

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [37]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    {gene_row_ids}
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    ['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209', 'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253', 'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262']
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['ILMN_1725881', 'ILMN_1910180', 'ILMN_1804174', 'ILMN_1796063', 'ILMN_1

In [38]:
if requires_gene_mapping:
    identifier_key = 'ID'
    gene_symbol_key = 'Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [39]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [40]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [41]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 133 samples.


In [42]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Anorexia-Nervosa', the least common label is '1.0' with 15 occurrences. This represents 11.28% of the dataset.
The distribution of the feature 'Anorexia-Nervosa' in this dataset is fine.

Quartiles for 'Age':
  25%: 30.043835
  50% (Median): 45.224657
  75%: 54.424657
Min: 16.183561
Max: 84.057534
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 36 occurrences. This represents 27.07% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [43]:
if is_available:
    save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')
else:
    save_cohort_info(cohort, JSON_PATH, is_available)

A new JSON file was created at: C:/Users/arthu/OneDrive/Documents/GitHub/output2\Arthur\Anorexia-Nervosa\cohort_info.json


In [44]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

### 3. Do regression & Cross Validation

In [50]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE60190,True,True,False,False,True,133,


In [51]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

In [52]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
